In [ ]:
# Function to crop and stack landsat08 images
def crop_stack_data(files_to_crop, crop_dir_path, crop_bound):
    """Crops a set of tif files and saves them 
    in a crop directory. Returns a stacked numpy 
    array of bands.
    
    Parameters
    ----------
    files_to_crop : list
        List of paths to multispectrum scenes 
        (.tiff) to be cropped.
    
    crop_dir_path : string
        The path to an output directory.  
        use conditional statment to create one,
        if not in existance. 
        The directory will store, 
        cropped and stacked bands.
        
    crop_bound : gpd (GeoPandas GeoDataFrame)
        Vector shape file geodataframe used for 
        cropping area of interest from the files_to_crop.
    
    Returns
    -------
    all_bands_stack : numpy array(s)
        Stacked and cropped numpy array bands 
        (our new aoi's).
    
    fire_crop_bound : GeoPandas GeoDataFrame
        A vector shape file that either shares 
        the crs of the stacked bands or is 
        reprojected from the crop_bound crs.
    """
    if not os.path.exists(crop_dir_path):
        os.mkdir(crop_dir_path)
        
    # Reproject boundary .shp to geotiff crs 
    with rio.open(files_to_crop[0]) as landsat_src:
        if not crop_bound.crs == landsat_src.crs:
            fire_crop_bound = crop_bound.to_crs(
                                  landsat_src.crs)
            extent = plotting_extent(landsat_src)
#         crop_bound = crop_bound.to_crs(src.crs)
 
    #  crop images using earthpy 
    es.crop_all(raster_paths=files_to_crop,
                output_dir=crop_dir_path,
                geoms= fire_crop_bound,
                overwrite=True)
    
    # Retrieve cropped bands from the new directory
    all_bands = sorted(glob(os.path.join(
                       crop_dir_path, "*.tif")))
    # Stack cropped bands
    all_stack_bands, stack_bands_meta = es.stack(all_bands)
    
    # return cropped, stacked bands and fire boundary
    return all_stack_bands, stack_bands_meta, fire_crop_bound

In [5]:
# Function to mask clouds from the landsat08 images
def mask_data(arr, path_to_qa):   
    """Function that masks numpy arrays 
    using qa layer.
    
    Parameters
    ----------
    arr : numpy array
        Numpy array(s) of bands of aoi       
    
    path_to_qa : str
        Path to QA layer(s) associated with each band        
    
    Returns
    -------
    arr : masked numpy array
        Updated numpy array(s) of bands with high cloud
        confidence, clouds and cloud shadows masked.        
    """
    # 1. open the qa layer
    with rio.open(path_to_qa[0]) as src:
        mask_arr = src.read(1)
        
    # Cloud mask values
    high_cloud_confidence = em.pixel_flags["pixel_qa"]["L8"]["High Cloud Confidence"]
    cloud = em.pixel_flags["pixel_qa"]["L8"]["Cloud"]
    cloud_shadow = em.pixel_flags["pixel_qa"]["L8"]["Cloud Shadow"]

    all_masked_values = cloud_shadow + cloud + high_cloud_confidence
    
    # 2. Mask the numpy array 
    if any(i in np.unique(mask_arr) for i in all_masked_values):
        landsat_masked_bands = em.mask_pixels(arr,
                                              mask_arr,
                                              vals=all_masked_values)
        return landsat_masked_bands
    else:
        print("There are no pixels to mask")
        return arr

In [6]:
# Function to classify burned areas
def classify_dnbr(arr):
    """Function that creates a new numpy array of classified
    values using difference normalized burn ratio (dNBR) as
    numpy array. 
    
    Parameters
    ----------
    arr : Numpy array
        Numpy array(s) containing dNBR data.        
    
    Returns
    -------
    arr_class : Numpy array
        Numpy array(s) containing reclassified 
        dNBR values in 5 possible classes.        
    """

    # YOUR CODE HERE
    # bins to define fire severity
    class_bins = [-np.inf, -.1, .1, .27, .66, np.inf]
    
    # code to classify dnbr values 
    arr_reclass = np.digitize(arr, class_bins)
        
    return arr_reclass